# Solving the Four-Player Intransitive Dice Problem

Intransitive (or Non-transitive) dice are dice which have a greater than 50% chance of winning in some kind of cyclical (rock-paper-scissors-like) pattern. For instance, Die A beats B, B beats C, and paradoxically enough, C beats A.  It seems unusual that such a thing could exist but, funnily enough, it can, and is [well documented](https://en.wikipedia.org/wiki/Intransitive_dice)

So the question is how many dice do you need such that you can you find a die that will always beat some selection of dice by some other players?

Let's say you have only one friend.   Any set of three intransitive dice will work, because if they pick C, you pick B, etc.  But what about if you have two friends?  The same set will force you to lose to one and win to the other.  But we want to beat both!  If you try adding in two dice to bring the total to five dice, you'll find that again, it's possible for your friends to pick two dice that have no common winner.  (In rock-paper-scissors-lizard-Spock terms, think if they pick Spock and scissors. Rock, paper, and lizard lose to Spock, scissors, and scissors respectively.)

It can be shown, however, that from a set of seven properly constructed dice, one can pull a die that beats any combination of two dice.  Thus the three-player game is solved with oscar's dice.

Can we make some combination that solves for four players?  Yes! (although I call that p=3 because I'm just interested in the adversarial players)

Let's look at the three-player version for some hints. Oscar's dice are three-sided dice (or doubled-up six-sided dice) that have some interesting properties.  I'll substract one off of all the faces to zero-index it, and show only the three-sided die version:

In [1]:
import numpy as np
oscar_dice = np.array([[0, 11, 19], [1, 13, 16], [2, 8, 20], [3, 10, 17], [4, 12, 14], [5, 7, 18], [6, 9, 15]])

There are a few things to notice:

- there's a low, medium and high face for each of the dice, and they are the numbers 0 through 6 offset by 7 each time.
- therefore we can just number each of the dice by the lowest face
- the total of each die is the same
- the number of faces is (n-1)/2
- the "middle die", is 3 mod 7 for all the faces
- the modulus is on rotation.  So it skips forward one on the lowest face, two on the highest, and four in the middle.  Take a look:

In [2]:
dict(zip('lowest middle highest'.split(),(oscar_dice%7).T.tolist()))

{'lowest': [0, 1, 2, 3, 4, 5, 6],
 'middle': [4, 6, 1, 3, 5, 0, 2],
 'highest': [5, 2, 6, 3, 0, 4, 1]}

We took the modulus, and the lowest has sequential values adjacent to each other, the highest, with one skip, and the middle with a skip forward of four (look a the positioning of 2 and 4 with respect to 3).

According to [Ried et al](https://www.researchgate.net/publication/266258217_Domination_and_irredundance_in_tournaments), the number of dice needed for a four-player domination tournament is 19.  I was not able to find a solution for 19, nor 21 (it works better for primes because the skipping-ahead trick shown above doesn't work for skips of 3 and 7).  But I found a solution for 23.  Observe:

In [3]:
import numpy as np
from collections import Counter
import itertools as it
from fractions import Fraction
from pprint import pprint

n = 23 #define the number of dice
p = 3 #define the number of adversaries

#define a function that shows which die wins
wins = lambda d1, d2: np.sign(sum([1 if i>j else 0 if i==j else -1 for i in d1 for j in d2]))

#define a function that shows by how much
bias = lambda d1, d2: sum([1 if i>j else 0 if i==j else -1 for i in d1 for j in d2])
has_winning_die = lambda s: Counter([j[0] for i in s for j in d if i!=j if wins(i,j)>0]).most_common(1)[0][1]==p


In [4]:
#for quick ref
r = list(range(n))

#this shows which rounds are available that are at constant skip values
rounds = [[i*k%n for i in r] for k in range(1,n)]

#these are all possible rounds that pass the test of being uniquely valued (that gcd(k,n) above is 1)
v = sorted([i for i in rounds if len(set(i))==n],key=lambda x:x.index(1))

#rotate the lists to the oscar dice form 
o = list(zip(*[i[i.index(n//2):]+i[:i.index(n//2)] for i in v]))

#what are the possible values? Make sure that the number of faces is odd, and correct for the tournament
c = next(k for k in range(1,n,2) if any(i for i in it.combinations(zip(*o),k) if len(set(map(sum,zip(*i))))==1))
assert c==(n-1)//2

#balanced values of the modulo that all add up (probably could have been done with the previous step)
b = [i for i in it.combinations(zip(*o),c) if len(set(map(sum,zip(*i))))==1]

#potential dice to check
t = [list(zip(*[[l+j*n for l in k] for j,k in enumerate(i)])) for i in b]

#make sure we didn't make any mistakes and they are uniquely valued
assert all([c*n==len(set([j for k in i for j in k])) for i in t])

#let's try the first one 
d = t[0] 

#assert that it works
assert not [q for q in it.combinations([[j[0] for j in d if i!=j if wins(i,j)>0] for i in d],p) if not [j for j in range(n) if all(j in k for k in q)]]

pprint(d)

[(11, 34, 57, 80, 103, 126, 149, 172, 195, 218, 241),
 (12, 23, 65, 86, 107, 129, 144, 174, 188, 208, 250),
 (13, 35, 50, 69, 111, 132, 139, 176, 204, 221, 236),
 (14, 24, 58, 75, 92, 135, 157, 178, 197, 211, 245),
 (15, 36, 66, 81, 96, 115, 152, 180, 190, 224, 231),
 (16, 25, 51, 87, 100, 118, 147, 182, 206, 214, 240),
 (17, 37, 59, 70, 104, 121, 142, 161, 199, 227, 249),
 (18, 26, 67, 76, 108, 124, 160, 163, 192, 217, 235),
 (19, 38, 52, 82, 112, 127, 155, 165, 185, 207, 244),
 (20, 27, 60, 88, 93, 130, 150, 167, 201, 220, 230),
 (21, 39, 68, 71, 97, 133, 145, 169, 194, 210, 239),
 (22, 28, 53, 77, 101, 136, 140, 171, 187, 223, 248),
 (0, 40, 61, 83, 105, 116, 158, 173, 203, 213, 234),
 (1, 29, 46, 89, 109, 119, 153, 175, 196, 226, 243),
 (2, 41, 54, 72, 113, 122, 148, 177, 189, 216, 252),
 (3, 30, 62, 78, 94, 125, 143, 179, 205, 229, 238),
 (4, 42, 47, 84, 98, 128, 138, 181, 198, 219, 247),
 (5, 31, 55, 90, 102, 131, 156, 183, 191, 209, 233),
 (6, 43, 63, 73, 106, 134, 151, 162, 184

In [5]:
#double check, what is the bias?
d1,d2 = d[:2]
print((lambda x: Fraction(sum(x),len(x)))([i<j for i in d1 for j in d2]))

62/121


In [6]:
#is this true for all values?
bias_matrix = np.array([[bias(i,j) for i in d] for j in d])
print(bias_matrix)

[[ 0  3  3  3  3 -3  3 -3  3  3 -3 -3  3  3 -3 -3  3 -3  3 -3 -3 -3 -3]
 [-3  0  3  3  3  3 -3  3 -3  3  3 -3 -3  3  3 -3 -3  3 -3  3 -3 -3 -3]
 [-3 -3  0  3  3  3  3 -3  3 -3  3  3 -3 -3  3  3 -3 -3  3 -3  3 -3 -3]
 [-3 -3 -3  0  3  3  3  3 -3  3 -3  3  3 -3 -3  3  3 -3 -3  3 -3  3 -3]
 [-3 -3 -3 -3  0  3  3  3  3 -3  3 -3  3  3 -3 -3  3  3 -3 -3  3 -3  3]
 [ 3 -3 -3 -3 -3  0  3  3  3  3 -3  3 -3  3  3 -3 -3  3  3 -3 -3  3 -3]
 [-3  3 -3 -3 -3 -3  0  3  3  3  3 -3  3 -3  3  3 -3 -3  3  3 -3 -3  3]
 [ 3 -3  3 -3 -3 -3 -3  0  3  3  3  3 -3  3 -3  3  3 -3 -3  3  3 -3 -3]
 [-3  3 -3  3 -3 -3 -3 -3  0  3  3  3  3 -3  3 -3  3  3 -3 -3  3  3 -3]
 [-3 -3  3 -3  3 -3 -3 -3 -3  0  3  3  3  3 -3  3 -3  3  3 -3 -3  3  3]
 [ 3 -3 -3  3 -3  3 -3 -3 -3 -3  0  3  3  3  3 -3  3 -3  3  3 -3 -3  3]
 [ 3  3 -3 -3  3 -3  3 -3 -3 -3 -3  0  3  3  3  3 -3  3 -3  3  3 -3 -3]
 [-3  3  3 -3 -3  3 -3  3 -3 -3 -3 -3  0  3  3  3  3 -3  3 -3  3  3 -3]
 [-3 -3  3  3 -3 -3  3 -3  3 -3 -3 -3 -3  0  3  3  3  3 -3  3 -3

In [7]:
#let's define a winning dictionary
w = {i[0]:sorted(j[0] for j in d if wins(i,j)>0) for i in sorted(d)}
pprint(w)

{0: [5, 7, 10, 11, 14, 15, 17, 19, 20, 21, 22],
 1: [0, 6, 8, 11, 12, 15, 16, 18, 20, 21, 22],
 2: [0, 1, 7, 9, 12, 13, 16, 17, 19, 21, 22],
 3: [0, 1, 2, 8, 10, 13, 14, 17, 18, 20, 22],
 4: [0, 1, 2, 3, 9, 11, 14, 15, 18, 19, 21],
 5: [1, 2, 3, 4, 10, 12, 15, 16, 19, 20, 22],
 6: [0, 2, 3, 4, 5, 11, 13, 16, 17, 20, 21],
 7: [1, 3, 4, 5, 6, 12, 14, 17, 18, 21, 22],
 8: [0, 2, 4, 5, 6, 7, 13, 15, 18, 19, 22],
 9: [0, 1, 3, 5, 6, 7, 8, 14, 16, 19, 20],
 10: [1, 2, 4, 6, 7, 8, 9, 15, 17, 20, 21],
 11: [2, 3, 5, 7, 8, 9, 10, 16, 18, 21, 22],
 12: [0, 3, 4, 6, 8, 9, 10, 11, 17, 19, 22],
 13: [0, 1, 4, 5, 7, 9, 10, 11, 12, 18, 20],
 14: [1, 2, 5, 6, 8, 10, 11, 12, 13, 19, 21],
 15: [2, 3, 6, 7, 9, 11, 12, 13, 14, 20, 22],
 16: [0, 3, 4, 7, 8, 10, 12, 13, 14, 15, 21],
 17: [1, 4, 5, 8, 9, 11, 13, 14, 15, 16, 22],
 18: [0, 2, 5, 6, 9, 10, 12, 14, 15, 16, 17],
 19: [1, 3, 6, 7, 10, 11, 13, 15, 16, 17, 18],
 20: [2, 4, 7, 8, 11, 12, 14, 16, 17, 18, 19],
 21: [3, 5, 8, 9, 12, 13, 15, 17, 18, 19, 

In [8]:
# define a function that tells you which die will beat any three dice that are given to you.
def which_wins(dice):
    return {i for i,j in w.items() if all(die in j for die in dice)}

print([which_wins((1,3,5)),
which_wins((1,2,3)),
which_wins((1,2,4)),
which_wins((1,3,4)),])

[{9, 7}, {4, 5}, {10, 5}, {5, 7}]


In [9]:
#make sure that all combinations of dice have a winning solution.
assert all(len(which_wins(i)) for i in it.combinations(range(n),p))

#one last sanity double-check with a one-liner that checks if there always exists some number that is on all three lists for every combination of dice, this number represents the lowest face of the die
assert all(Counter([k[0] for j in i for k in d if sum([(i1<i2)-(i1>i2) for i1,i2 in zip(j,k)])>0]).most_common(1)[0][1]==p for i in it.combinations(d,3))

There you have it! 11-sided dice that have a 62/121 chance to beat the next die on the list.  That's a 51% but if it works for roulette tables at the casino, it works for me.